In [40]:
import pandas as pd
import cv2
from transformers import CLIPProcessor, CLIPModel
import os
import ast
from PIL import Image
from deep_translator import GoogleTranslator
import matplotlib.pyplot as plt

## Extraction des images détectées par le Yolo dans des fichiers séparés ##

In [ ]:
def extract_images(fichier, chemin_image):
    df=pd.read_csv(fichier)
    liste_coords =df['box'].to_list()
        
    path=os.getcwd()
    
    image = cv2.imread(path + "\\"+ chemin_image)
    
    if image is None:
            print(f"Erreur : impossible de charger l'image {chemin_image}")
            exit()
    nom_dossier = chemin_image.replace(".jpg","").replace('images_a_traiter\\','')
    print(nom_dossier)
    if not os.path.exists(path+"\\"+"sorties"+"\\"+nom_dossier):
        os.makedirs("sorties\\"+nom_dossier)
        print(f"{nom_dossier} créé")
    compteur=0
    for i in liste_coords:
        compteur+=1
        dico = ast.literal_eval(i)
        valeurs=list(dico.values())
        
        y1, x1, y2, x2 = int(round(valeurs[0])), int(round(valeurs[1])), int(round(valeurs[2])), int(round(valeurs[3]))
        cropped_img= image[x1:x2,y1:y2]
        cv2.imwrite(f"sorties\\{nom_dossier}\\{nom_dossier}_illus{compteur}.png", cropped_img)
            


In [27]:
path=os.getcwd()+"\\images_a_traiter"
for root, dirs, file in os.walk(path):
    for image in file:
        if ".jpg" in image:
            nom_csv = image+"_annots.csv"
            chemin_image = "images_a_traiter\\"+image
            extract_images(nom_csv, chemin_image)

Excelsior19240710image1
Excelsior19240710image1 créé
Excelsior19240710image4
Excelsior19240710image4 créé


Fonction optionnelle pour afficher les images d'une page sous forme de grille, empruntée à HALUKPA Kerry, "Getting started with OpenAI's CLIP", sur Medium, 2023, consulté le 27/08/2025.
https://medium.com/@kerry.halupka/getting-started-with-openais-clip-a3b8f5277867

In [ ]:

def image_grid(imgs, cols):
    rows = (len(imgs) + cols - 1) // cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


def montrer_images(nom_image):
    path=os.getcwd()

    image_urls=[]
    for i in range(1,20):
        image_urls.append(str(path+"\\"+"sorties"+"\\"+nom_image+"_illus"+str(i)+".png"))
    images = []
    try:
        for url in image_urls:
            images.append(Image.open(url))
    except:
        pass

    grid = image_grid(images, cols=3)
    display(grid)

In [53]:
def clip_integral():
    liste_images = obtenir_liste_images()
    for i in liste_images:
        liste2=obtenir_liste_captions(i)
        liste3 = traduire(liste2)
        application_CLIP(liste3, i, liste2)

In [54]:
def obtenir_liste_images():
    path=os.getcwd()+"\\sorties"
    liste_images = []
    for root, dirs, files in os.walk(path):
            
        for dir in dirs:
                
            liste_images.append(str(dir))
    return(liste_images)

In [55]:
def obtenir_liste_captions(nom_image):
   fichier_texte=str('dossier_output_experience\\texte'+nom_image+".jpg.txt") #modifier le .jpg selon le format de l'image de la page de journal originale
   with open (fichier_texte,'r', encoding="utf-8") as ft:
         texte=ft.read()
         liste_zones=texte.split("Zone_texte")
         
         liste2=[]
         for i in range(0, len(liste_zones)):
            
            if len(liste_zones[i])>= 10 and len(liste_zones[i])<=77:
               terme=liste_zones[i].replace("\n","")
               terme=liste_zones[i].replace("- ","")
               liste2.append(terme)
   return(liste2)       


In [56]:
def traduire(liste2):
    liste3= GoogleTranslator("fr","en").translate_batch(liste2)
    return(liste3)
#Cellule pour passer les textes en anglais, CLIP ayant été entraîné sur des textes en anglais.

In [ ]:
def application_CLIP(liste3, dossier, liste2):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    images=[]
    path= os.getcwd()+"\\sorties"+"\\"+dossier
    for root, dirs, files in os.walk(path):
        for image in files:
            if ".png" in image:
                image= Image.open(path+"\\"+image)
                images.append(image)
        
    inputs = processor(text=liste3, images=images, return_tensors="pt", padding=True, do_rescale= True, do_convert_rgb=False)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
    liste_probas =probs.tolist()
    liste_probas_par_ordre_images=[]

    for i in range(0,len(liste_probas)):
        indices_probas=[]
        for probab in liste_probas[i]:
            if probab>=0.4:
                indices_probas.append(liste_probas[i].index(probab))
        liste_probas_par_ordre_images.append(indices_probas)
    compteur=1
    for i in liste_probas_par_ordre_images:
        print(f"image n{compteur}")
        compteur+=1
        for y in i:
            print(liste2[y])
    return(probs, liste_probas_par_ordre_images)        

In [72]:
clip_integral()

image n1
 JUILLET 1924 

 LES EPREUVES OLYMPIQUES AU STADE DE COLOMBES 

image n2
 JUILLET 1924 

 L'ancien président de la Répu" aspire au repos blique „ de la campagne 

image n3
 JUILLET 1924 

image n4
 JUILLET 1924 

image n5
 JUILLET 1924 

image n6
 JUILLET 1924 

image n7
 L'ancien président de la Répu" aspire au repos blique „ de la campagne 

 Un adversaire du projet 

image n8
 JUILLET 1924 

image n9
 JUILLET 1924 

 LES EPREUVES OLYMPIQUES AU STADE DE COLOMBES 

image n10
 JUILLET 1924 

 LES EPREUVES OLYMPIQUES AU STADE DE COLOMBES 



In [ ]:

def repres_probs(probs):
    fig = plt.figure(figsize=(8, 20))

    for idx in range(len(images)):

        # show original image
        fig.add_subplot(len(images), 2, 2*(idx+1)-1 )
        plt.imshow(images[idx])
        plt.xticks([])
        plt.yticks([])

        # show probabilities
        fig.add_subplot(len(images), 2, 2*(idx+1))
        
        plt.barh(range(len(probs[0].detach().numpy())),probs[idx].detach().numpy(), tick_label=liste2)
        plt.xlim(0,1.0)

        plt.subplots_adjust(left=0.1,
                            bottom=0.1,
                            right=0.9,
                            top=0.9,
                            wspace=0.2,
                            hspace=0.8)

    plt.show()

In [ ]:
len(probs)
liste_probas =probs.tolist()

In [ ]:
print (liste_probas)
for i in liste_probas:
    print(i)

In [ ]:
probabs_image1=[]
for probab in liste_probas[0]:
    if probab>=0.4:
        probabs_image1.append(liste_probas[0].index(probab))
print(probabs_image1)

[9]


In [ ]:
liste_probas_par_ordre_images=[]

for i in range(0,len(liste_probas)):
    indices_probas=[]
    for probab in liste_probas[i]:
        if probab>=0.2:
            indices_probas.append(liste_probas[i].index(probab))
    liste_probas_par_ordre_images.append(indices_probas)


Dans la cellule ci dessous, l'indice correspond au numéro de l'image, les chiffres à l'indice dans la liste des morceaux de texte des textes associés à plus de 0,2.

In [ ]:
print(liste_probas_par_ordre_images)

In [ ]:
compteur=1
for i in liste_probas_par_ordre_images:
    print(f"image n{compteur}")
    compteur+=1
    for y in i:
        print(liste2[y])